In [7]:
import pandas as pd

df = pd.read_parquet('/home/jovyan/MGTA 415/project/AmazonReviews2023/Amazon-Reviews-2023 database/raw_meta_Cell_Phones_and_Accessories/full-00000-of-00007.parquet')
df.head()

,main_category,title,average_rating,rating_number,features,description,price,images,videos,store,categories,details,parent_asin,bought_together,subtitle,author
0,Cell Phones & Accessories,ARAREE Slim Diary Cell Phone Case for Samsung ...,3.8,5,"[Genuine Cow leather with 6 different colors, ...","[JUST LOOK, You can tell the difference. Make ...",None,"{'hi_res': [None, None, None, None, None, None...","{'title': [], 'url': [], 'user_id': []}",araree,"[Cell Phones & Accessories, Cases, Holsters & ...","{""Product Dimensions"": ""3.35 x 0.59 x 6.18 inc...",B013SK1JTY,None,None,None
1,Cell Phones & Accessories,Bastmei for OnePlus 7T Case Extremely Light Ul...,4.4,177,[Ultra-thin & Ultra-light: The ultra slim fit ...,[],11.98,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Bastmei,"[Cell Phones & Accessories, Cases, Holsters & ...","{""Package Dimensions"": ""7.6 x 4.29 x 0.75 inch...",B07ZPSG8P5,None,None,None
2,Cell Phones & Accessories,Wireless Fones Branded New Iphone 5C/LITE Hot ...,4.0,2,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",WIRELESS FONES,"[Cell Phones & Accessories, iPhone Accessories]","{""Item model number"": ""Apple Iphone 5C"", ""Othe...",B00GKR3L12,None,None,None
3,Cell Phones & Accessories,"iPhone 6 Plus + Case, DandyCase Perfect PATTER...",4.0,15,"[Slim-Fit design for the iPhone 6 Plus (5.5"" s...",[Case does not need to be removed for charging...,None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",DandyCase,"[Cell Phones & Accessories, iPhone Accessories]","{""Product Dimensions"": ""5.43 x 0.28 x 2.64 inc...",B00PB8U8BW,None,None,None
4,Cell Phones & Accessories,"Case for Galaxy S6/S6 Edge, Thin Translucent V...",4.0,1,[],[],None,"{'hi_res': [None, None], 'large': ['https://m....","{'title': [], 'url': [], 'user_id': []}",7Pite,"[Cell Phones & Accessories, Cases, Holsters & ...","{""Package Dimensions"": ""8.31 x 3.74 x 0.55 inc...",B07D3RHSRV,None,None,None


In [ ]:
## No need this for now (still trying to figure out why there is no image)

import ast

def has_valid_image(imgs):
    try:
        # ✅ If it is a string, try to convert it into a dictionary first
        if isinstance(imgs, str) and imgs.strip().startswith("{"):
            imgs = ast.literal_eval(imgs)

        # ✅ If it is successfully converted into a dict, continue to judge
        if isinstance(imgs, dict):
            for key in ['hi_res', 'large']:
                urls = imgs.get(key, [])
                if isinstance(urls, list) and any(url.startswith("http") for url in urls):
                    return True
        return False
    except:
        return False

df['has_image'] = df['images'].apply(has_valid_image)
print("✅ Number of rows with images：", df['has_image'].sum())



✅ 有图片的行数： 0


In [ ]:
def clean_for_search(df):
    print("💡 Number of original data rows：", len(df))

    # Safe concatenation list or other string
    def safe_join(x):
        try:
            if isinstance(x, list):
                return ' '.join(map(str, x))
            return str(x)
        except:
            return ''

    # Only keep the data with "something" in title
    df = df[df['title'].notnull()]
    df['title'] = df['title'].astype(str).str.strip()

    df['description'] = df['description'].apply(safe_join)
    df['features'] = df['features'].apply(safe_join)

    df['price_display'] = df['price'].apply(lambda x: f"${x:.2f}" if x > 0 else "N/A")

    df['image_url'] = df['images'].apply(
    lambda x: x['large'][0] if isinstance(x, dict) and 'large' in x and isinstance(x['large'], list) and len(x['large']) > 0 else None)

    df['image_url'] = df['image_url'].fillna('https://via.placeholder.com/300x300.png?text=No+Image')

    print("✅ Reserved fields：title, description, features, price, image_url")

    return df[['title', 'description', 'features', 'price', 'image_url']]



In [ ]:
import pandas as pd
import requests

def check_image_url(url):
    try:
        response = requests.head(url, timeout=3)
        return response.status_code == 200
    except:
        return False

df = pd.read_csv("cleaned_amazon_products.csv")

df['image_ok'] = df['image_url'].apply(check_image_url)

sample_df = df.sample(200, random_state=42)
sample_df['image_ok'] = sample_df['image_url'].apply(check_image_url)

print("✅ Valid number of pictures：", sample_df['image_ok'].sum())
print("❌ Invalid image count：", (~sample_df['image_ok']).sum())



In [ ]:
cleaned_df = clean_for_search(df)
print("✔️ Function return result type：", type(cleaned_df))
cleaned_df.head(10)



💡 原始数据行数： 184070
✅ 清洗完成，保留字段：title, description, features, price, image_url
✔️ 函数返回结果类型： <class 'pandas.core.frame.DataFrame'>


,title,description,features,price,image_url
0,ARAREE Slim Diary Cell Phone Case for Samsung ...,"[""JUST LOOK, You can tell the difference. Make...",['Genuine Cow leather with 6 different colors'...,-1.00,https://yourdomain.com/placeholder.jpg
1,Bastmei for OnePlus 7T Case Extremely Light Ul...,[],['Ultra-thin & Ultra-light: The ultra slim fit...,11.98,https://yourdomain.com/placeholder.jpg
2,Wireless Fones Branded New Iphone 5C/LITE Hot ...,[],[],-1.00,https://yourdomain.com/placeholder.jpg
3,"iPhone 6 Plus + Case, DandyCase Perfect PATTER...",['Case does not need to be removed for chargin...,"['Slim-Fit design for the iPhone 6 Plus (5.5"" ...",-1.00,https://yourdomain.com/placeholder.jpg
4,"Case for Galaxy S6/S6 Edge, Thin Translucent V...",[],[],-1.00,https://yourdomain.com/placeholder.jpg
5,Rikki Knight 3D Chevron Peach on White with An...,['The Rikki Knight new Flip Wallet iPhone case...,['Includes attached ribbon Wristlet Strap.'\n ...,22.42,https://yourdomain.com/placeholder.jpg
6,"iMangoo Galaxy S22 Case, for Samsung Galaxy S2...",[],['Compatibility: This Galaxy S22 case is only ...,11.99,https://yourdomain.com/placeholder.jpg
7,Piel Frama Wallet Case for Samsung Galaxy Nexu...,"['Handcrafted in Spain by Leather artisans, pi...",['Sync through travel cable' 'Removable belt c...,-1.00,https://yourdomain.com/placeholder.jpg
8,Pearl Bowknot 3d Cute Crystal Bling rhinestone...,[],[],-1.00,https://yourdomain.com/placeholder.jpg
9,Omio for Galaxy Note 10 Plus Privacy Screen Pr...,[],[],-1.00,https://yourdomain.com/placeholder.jpg


In [58]:
cleaned_df.to_csv("cleaned_amazon_products.csv", index=False)